In [1]:
import rag_utils

c:\Users\voyno\Desktop\code\repos\repo-chat\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
repo = "https://github.com/hwchase17/langchain"
vectorstore = rag_utils.create_vectorstore(repo)
qa = rag_utils.get_retrieval_chain(vectorstore)

Fetching data from git repo...
Clearing index testindex from previous runs...


  0%|          | 0/1142 [00:00<?, ?it/s]

Done!
{'dimension': 1536,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 4196}},
 'total_vector_count': 4196}


In [24]:
query = "How can I use this code?"

similarity_resp = vectorstore.similarity_search_with_score(
    query, k=5
)
response = qa.run(query)

In [25]:
print(f"\nQuery:\n{query}")
print(f"\nResponse:\n{response}")
for doc, score in similarity_resp:
    print(f"\nSimilarity Response (Score: {score:.2f})\n{doc.page_content[:100]}")


Query:
How can I use this code?

Response:
I'm not sure which code you are referring to. Can you please provide more context or specify which code you are asking about?

Similarity Response (Score: 0.78)
5. Ask questions: Define a list of questions to ask about the codebase, and then use the Conversatio

Similarity Response (Score: 0.77)
Around this time, in the spring of 2000, I had an idea. It was clear from our experience with Viaweb

Similarity Response (Score: 0.76)
chain.run("some initial text")
```

See the [Quick Start](https://graphsignal.com/docs/guides/qui

Similarity Response (Score: 0.76)
** helppage|help-mediawiki
* SEARCH
* TOOLBOX</text>
      <sha1>qqsu3aocmn2qji3pfn56y3pizazd8jv<

Similarity Response (Score: 0.76)
<html>
  <head>
    <title>Chew dad's slippers</title>
  </head>
  <body>
    <h1>
      Inste
